In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

from itertools import cycle, combinations

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
#data.sample(5)

In [3]:
#data.describe()

# Предобработка

In [4]:
def count_genres(genres_series_list):
    cnt = Counter()
    for genre_list in genres_series_list:
        for genre in genre_list:
            cnt[genre] += 1
    return cnt


answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:
data['profit'] = data['revenue'] - data['budget']
data["cast"] = data["cast"].apply(lambda x : x.split("|"))
data["genres"] = data["genres"].apply(lambda x : x.split("|"))
data["director"] = data["director"].apply(lambda x : x.split("|"))
data["production_companies"] = data["production_companies"].apply(lambda x : x.split("|"))
data["release_date"] = data["release_date"].apply(lambda x : x.split("/"))


#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

# 1. У какого фильма из списка самый большой бюджет?

In [5]:
max_budget_df = data[data.budget == data['budget'].max()]
film_name = max_budget_df['original_title'].iloc[0]
film_id = max_budget_df['imdb_id'].iloc[0]
result = "{} {}".format(film_name,film_id)

answers['1'] = result

# 2. Какой из фильмов самый длительный (в минутах)?

In [6]:
max_runtime_df = data[data.runtime == data['runtime'].max()]
film_name = max_runtime_df['original_title'].iloc[0]
film_id = max_runtime_df['imdb_id'].iloc[0]
result = "{} {}".format(film_name,film_id)

answers['2'] = result

# 3. Какой из фильмов самый короткий (в минутах)?





In [7]:
min_runtime_df = data[data.runtime == data['runtime'].min()]
film_name = min_runtime_df['original_title'].iloc[0]
film_id = max_runtime_df['imdb_id'].iloc[0]
result = "{} {}".format(film_name,film_id)

answers['3'] = result

# 4. Какова средняя длительность фильмов?


In [8]:
mean_film_runtime = data['runtime'].mean()

answers['4'] = round(mean_film_runtime)

# 5. Каково медианное значение длительности фильмов? 

In [9]:
median_film_runtime = data['runtime'].median()

answers['5'] = round(median_film_runtime)

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [10]:
max_profit_df = data[data.profit == data['profit'].max()]
film_name = max_profit_df['original_title'].iloc[0]
film_id = max_profit_df['imdb_id'].iloc[0]
result = "{} {}".format(film_name,film_id)

answers['6'] = result 

# 7. Какой фильм самый убыточный? 

In [11]:
min_profit_df = data[data.profit == data['profit'].min()]
film_name = min_profit_df['original_title'].iloc[0]
film_id = min_profit_df['imdb_id'].iloc[0]
result = "{} {}".format(film_name,film_id)

answers['7'] = result 

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [12]:
paid_off_df = data.query("revenue > budget")

answers['8'] = len(paid_off_df)

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [13]:
in2008_df = data.query("release_year == 2008")
max_df = in2008_df[in2008_df.profit == in2008_df['profit'].max()]

film_name = max_df['original_title'].iloc[0]
film_id = max_df['imdb_id'].iloc[0]
result = "{} {}".format(film_name,film_id)

answers['9'] = result

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [14]:
in2012_2014_df = data.query("2012 <= release_year <= 2014")
min_df = in2012_2014_df[in2012_2014_df.profit == in2012_2014_df['profit'].min()]

film_name = min_df['original_title'].iloc[0]
film_id = min_df['imdb_id'].iloc[0]
result = "{} {}".format(film_name,film_id)

answers['10'] = result

# 11. Какого жанра фильмов больше всего?

In [15]:
raw_genres = data["genres"]
counter = count_genres(raw_genres.to_list())

answers['11'] = counter.most_common(1)[0][0]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [35]:
exploded = data.explode("genres")
data_profit = exploded.query("profit > 0")
genres_profit = data_profit.groupby(["genres"]).sum()
max_profit_genres = genres_profit["profit"].sort_values(ascending = False)
answers['12'] = max_profit_genres.index[0]

answers['12']

'Adventure'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [17]:
director_expoloded = data.explode("director")
positive_profit = director_expoloded.query("profit > 0")
director_profit = positive_profit.groupby(["director"]).sum()
max_profit_directors = director_profit["profit"].sort_values(ascending = False)

answers['13'] = max_profit_directors.index[0]

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [18]:
genres_exploded = data.explode("genres")
action_genre = genres_exploded.query("genres == \"Action\"")
directors_explode = action_genre.explode("director")
directors = directors_explode["director"].value_counts()

answers['14'] = directors.index[0]

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [19]:
actors_exploded = data.explode("cast")
in_2012 = actors_exploded.query("release_year == 2012 & profit > 0")
profit_actors = in_2012.groupby(["cast"]).sum()['profit'].sort_values(ascending = False).head(5)

answers['15'] = profit_actors.index[0]

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?
Примечание: в фильмах, где бюджет выше среднего по данной выборке.

In [20]:
actors_exploded = data.explode("cast")
mean_budget = actors_exploded["budget"].mean()
above_mean_budget = actors_exploded[actors_exploded.budget > mean_budget]
actors = above_mean_budget["cast"].value_counts().head(5)

answers['16'] = actors.index[0]

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [21]:
actors_exploded = data.explode("cast")
nicolas = actors_exploded.query("cast == \"Nicolas Cage\"")
nicolas_genres = nicolas.explode("genres")["genres"].value_counts().head(5)

answers['17'] = nicolas_genres.index[0]

# 18. Самый убыточный фильм от Paramount Pictures

In [22]:
companies = data.explode("production_companies")
paramount = companies.query("production_companies == \"Paramount Pictures\"")
paramount_films = paramount.sort_values('profit').head()

film_name = paramount_films['original_title'].iloc[0]
film_id = paramount_films['imdb_id'].iloc[0]

answers['18'] = "{} ({})".format(film_name,film_id)

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [23]:
years_revenue = data.groupby(["release_year"]).sum()
sort = years_revenue.sort_values("revenue", ascending = False).head(5)

answers['19'] = sort.index[0]

# 20. Какой самый прибыльный год для студии Warner Bros?

In [24]:
companies = data.explode("production_companies")
warner = companies[companies.production_companies.str.contains("Warner")]
years_profit = warner.groupby(["release_year"]).sum()
sort = years_profit.sort_values("profit", ascending = False).head(5)

answers['20'] = sort.index[0]

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [25]:
movies = data
movies["month"] = movies["release_date"].apply(lambda x: x[0]) 
movies_pivot = movies.pivot_table(columns = "month", index = "original_title", values = "imdb_id",
                                 aggfunc = "count", fill_value = 0)
months = movies_pivot.sum().sort_values(ascending=False).head(3)

answers['21'] = months.index[0]

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [26]:
movies = data
movies["month"] = movies["release_date"].apply(lambda x: x[0])
summer = movies[movies["month"].isin(["6", "7", "8"])]

answers['22'] = len(summer["month"])

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [27]:
directors_explode = data.explode("director")
directors_explode["month"] = directors_explode["release_date"].apply(lambda x: x[0])
winter = directors_explode[directors_explode["month"].isin(["12", "1", "2"])]
directors = winter["director"].value_counts().head(5)

answers['23'] = directors.index[0]

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [28]:
companies = data.explode("production_companies")
companies["title_len"] = companies["original_title"].apply(lambda title: len(title))
max_title = companies.query("title_len == title_len.max()")

answers['24'] = max_title["production_companies"].iloc[1]

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [29]:
companies = data.explode("production_companies")
companies["overview_words"] = companies["overview"].str.split().apply(len)
mean = companies.groupby("production_companies")["overview_words"].mean().sort_values().tail(1)

answers['25'] = mean.index[0]

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [30]:
one_percent = data.sort_values('vote_average').tail(int(data.original_title.count()/100)).original_title
answers['26'] = one_percent.tolist()

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [31]:
raw_df = pd.read_csv('movie_bd_v5.csv')
raw_df['cast'] = raw_df['cast'].str.split('|')
cast_df = raw_df['cast'].reset_index()
cast_df['pairs'] = cast_df['cast'].apply(lambda cast_list: list(combinations(cast_list, 2)))
exploded_pairs = cast_df.explode('pairs').reset_index()

answers['27'] = Counter(exploded_pairs['pairs']).most_common(1)[0][0]

# Submission

In [32]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides tt1298650',
 '2': 'Gods and Generals tt0279111',
 '3': 'Winnie the Pooh tt0279111',
 '4': 110,
 '5': 107,
 '6': 'Avatar tt0499549',
 '7': 'The Lone Ranger tt1210819',
 '8': 1478,
 '9': 'The Dark Knight tt0468569',
 '10': 'The Lone Ranger tt1210819',
 '11': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': '9',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': ['The Theory of Everything',
  'The Lord of the Rings: The Two Towers',
  'There Will Be Blood',
  'Mr. Nobody',
  '12 Years a Slave',
  'The Pianist',
  'Gone Girl',
  'Memento',
  'Guardians of the Galaxy',
  'Inception',
  'The Wolf of Wall Street',
  'The Grand Budapest Hotel',
  'The Lord of the Rings: The Return of the King',
  'Room',
  'Interstellar',
  'Inside Out',


In [33]:
len(answers)

26